In [ ]:
import seaborn as sns

names = list(corr_fred.index)
corr_ordered_names = list()

prior=False

while(len(corr_ordered_names)!=len(names)):

    if(prior):
        subset = corr_fred.loc[names][[not_remove]]
    else:
        subset = corr_fred.loc[names][names]
    
    if(len(subset)==1):
        corr_ordered_names.append(subset.index[0])
        names.remove(subset.index[0])
        break        
        
    high_value = np.array(subset).ravel()[np.nanargmax(np.array(subset))]
    a_ = subset.where(subset==high_value).dropna(how='all').dropna(axis=1)
    
    name_set = [*a_.columns,*a_.index]
    
    a = pd.concat([subset[[name_set[0]]],subset.drop(name_set[0],axis=1)],axis=1).max().max()
    b = pd.concat([subset[[name_set[1]]],subset.drop(name_set[1],axis=1)],axis=1).max().max()
    
    #remove = []
    if(a>b):
        #print(name_set[0]+">"+name_set[1])
        #print('front:',name_set[0])
        corr_ordered_names.append(name_set[0])
        remove = name_set[0]
        not_remove = name_set[1]
        
    else:
        #print(name_set[0]+"<"+name_set[1])
        #print('front:',name_set[1])
        corr_ordered_names.append(name_set[1])
        #names.remove(name_set[1])
        remove = name_set[1]
        
        #prior depemdpmt on
        not_remove = name_set[0]        
    
    names.remove(remove)
    print(corr_ordered_names)
    prior=True
    
plt.figure(figsize=(11,8))
sns.heatmap(corr_fred.loc[corr_ordered_names][corr_ordered_names],cmap="Greens",annot=True)
plt.show()

In [ ]:
abs(corr_fred).to_csv('..\\data\\processed\\abs_corr_fred.csv')

temp_corr_melt = pd.melt(abs(corr_fred),ignore_index=False).reset_index().dropna().sort_values(by='value',ascending=False)
index_names = []
for t_ in range(0,len(temp_corr_melt)):
    t = temp_corr_melt.iloc[[t_]]
    my_string = t['index'].values[0]+' '+t['variable'].values[0]
    words = my_string.split()
    
    # sort() will sort the strings.
    words.sort()
    index_names.append(words[0]+' '+words[1])
    #print(words)

temp_corr_melt.index=index_names

temp_corr_no_dup = temp_corr_melt[~temp_corr_melt.index.duplicated(keep='first')]    

temp_corr_no_dup.to_csv('..\\data\\processed\\temp_corr_melt.csv')


In [ ]:
for t_ in range(0,len(temp_corr_no_dup.index)):
    #t = temp_corr_no_dup.index
    
    head_ = temp_corr_no_dup.iloc[t_:].head(1)
    
    my_string = head_['index'].values[0]+' '+head_['variable'].values[0]
    words = my_string.split()
    
    words.sort()
    
    left = words[0]
    right = words[1]

    #print(head_)
    display(head_)
    subset = temp_corr_no_dup['index']==left
    print(left)
    print(right)
    subset_w_dups = pd.concat([temp_corr_no_dup[(temp_corr_no_dup['index']==left) | (temp_corr_no_dup['index']==right)],temp_corr_no_dup[(temp_corr_no_dup['variable']==left) | (temp_corr_no_dup['variable']==right)]],axis=0)

    subset_w_dups[~subset_w_dups.index.duplicated(keep='first')]
        
    first = 

In [ ]:
temp_corr_no_dup.sort_values(by='value',inplace=True,ascending=False)
top = temp_corr_no_dup.head(1)

for i in range(0,100):
    left = top[['index']].values[0][0]
    right = top[['variable']].values[0][0]

    my_string = left+' '+right
    words = my_string.split()

    # sort() will sort the strings.
    words.sort()
    index_ = words[0]+' '+words[1]

    top = pd.concat([temp_corr_no_dup[temp_corr_no_dup['variable']==right],temp_corr_no_dup[temp_corr_no_dup['index']==left]],axis=0).sort_values(by='value',ascending=False).drop(index_).head(1)
    print(top.index)

In [ ]:
names = list(corr_fred.index)
sorted_ = []
corr_ordered_names = list()

while(len(names)!=0):
    sorted_corr_fred = abs(corr_fred.loc[names][names]).max().sort_values(ascending=False)
    print(sorted_corr_fred.head(1))
    #print(sorted_corr_fred.head(1).columns)
    head_ = sorted_corr_fred.head(1).index
    sorted_.append(head_[0])
    
    best_to_follow = abs(corr_fred).idxmax().loc[head_].values[0]
    print(best_to_follow)

    names.remove(head_)

In [ ]:
abs(corr_fred).loc[sorted_][sorted_]

In [ ]:
#used to compare against index's?  pointless to do a pcorr unless trying to reduce vars

def return_model_vars(data):
    #print(x)
    
    names = data.columns
    #print(names)
    
    #data = data[names].sort_values(kind="quicksort", by=names[0], ascending=True,key=abs).copy()
    #y = names[0]
    
    loop=True
    
    removes = []

    while(loop==True):

        #x_names = data.columns[~data.columns.isin([item for item in [y,*removes]])]

        #x_scores = data[[y,*x_names]].pcorr()[[y]]

        x_scores = data[x_names].pcorr()
        pd.concat([data[y],pd.DataFrame(whiten(np.array(data[x_names])),columns=x_names).set_index(data.index)],axis=1).pcorr()[[y]]

        #print(x_scores.loc[~x_scores.index.isin([y])])

        #print(x_names)

        index_set = data.index

        test_results = x_scores

        n = len(data)
        #n = 50
        df_ = n - 2

        t=abs(test_results)*np.sqrt(df_)/np.sqrt(1-abs(test_results)**2)

        test_results_least = t.iloc[[np.argmin(abs(t))]].index[0]

        t_score_least = t.iloc[[np.argmin(abs(t))]]

        crit_t = scipy.stats.t.ppf(1 - .05 / 2, df_)

        if(np.isnan(t_score_least.values[0][0])):
            remove = test_results_least
            removes.append(remove)

        if((t.loc[test_results_least]<crit_t)[0]):
            remove = test_results_least
            #print(remove,t_score_least.values[0][0])
            removes.append(remove)

        if((t.loc[test_results_least]>crit_t)[0]):
            loop=False

        #if type(x_names) == type(None):
            #loop=False
        if(len(x_names)==1):
            loop=False